In [ ]:
import pandas as pd
df=pd.read_csv('Automobile Insurance claim amount.csv')


In [ ]:
df

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#Lets Check the null values
df.isnull().sum()

In [ ]:
#Their is no nul value present in the dataset

In [ ]:
#Lets check the datatypes of the columns 
df.dtypes

In [ ]:
df.info()

In [ ]:
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split

In [ ]:
sns.heatmap(df.isnull())

In [ ]:
df.describe()

In [ ]:
#1) The mean is more than the median(50th percentile in all columns). 
#2)Their is large difference in 75% and max in claim amount,Income,Monthly Premium Auto and the Total Claim Amount. 


In [ ]:
dfcor=df.corr()
dfcor

In [ ]:
sns.heatmap(dfcor)

In [ ]:
#From the graph we observe that their is mostly negative correlation among the columns 
#Monthly Premium Auto and Total Claim Amount are little bit corrleated.

In [ ]:
df.columns 

In [ ]:
sns.violinplot(x='Claim Amount',data=df)
plt.show()

In [ ]:
#From the garph it represent that mostly claim amount was between the 500 to 1500

In [ ]:
sns.distplot(df['Claim Amount'],bins=20)
plt.show()

In [ ]:
sns.barplot(x='Vehicle Class',y='Total Claim Amount',data=df)
plt.show()

In [ ]:
#From this graph it clearly shows that Luxury car and the Luxury SUV have the highest number of total claim amount 

In [ ]:
sns.catplot(x='Vehicle Class',y='Total Claim Amount',data=df)

In [ ]:
# Again i am checking the datatype so that i their is any other datatype of string i will convert it into the other datatype
df.dtypes

In [ ]:
df.columns

In [ ]:
# I am using the Label Encoder to change the datatype of the particular columns so that i can work futher 
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Customer']=le.fit_transform(df['Customer'])
df['Country']=le.fit_transform(df['Country'])
df['State Code']=le.fit_transform(df['State Code'])
df['State']=le.fit_transform(df['State'])
df['Response']=le.fit_transform(df['Response'])
df['Coverage']=le.fit_transform(df['Coverage'])
df['Education']=le.fit_transform(df['Education'])
df['Effective To Date']=le.fit_transform(df['Effective To Date'])
df['EmploymentStatus']=le.fit_transform(df['EmploymentStatus'])
df['Gender']=le.fit_transform(df['Gender'])
df['Location Code']=le.fit_transform(df['Location Code'])
df['Marital Status']=le.fit_transform(df['Marital Status'])
df['Policy Type']=le.fit_transform(df['Policy Type'])
df['Policy']=le.fit_transform(df['Policy'])
df['Claim Reason']=le.fit_transform(df['Claim Reason'])
df['Sales Channel']=le.fit_transform(df['Sales Channel'])
df['Vehicle Class']=le.fit_transform(df['Vehicle Class'])
df['Vehicle Size']=le.fit_transform(df['Vehicle Size'])

In [ ]:
df.skew()

In [ ]:
# Treating using log
import numpy as np
for col in df.columns:
    if df.skew().loc[col]>0.55:
        df[col]=np.log1p(df[col])

In [ ]:
df.skew()

In [ ]:
df['Total Claim Amount'].plot.box()

In [ ]:
df['Vehicle Class'].plot.box()

In [ ]:
df['Income'].plot.box()

In [ ]:
collist=df.columns.values
ncol=26
nrows=10

In [ ]:
plt.figure(figsize=(ncol,5*ncol))
for i in range(1,len(collist)):
    plt.subplot(nrows,ncol,i+1)
    sns.boxplot(df[collist[i]],color='green',orient='v')
    plt.tight_layout()

In [ ]:
#Lets check for outliers
from scipy.stats import zscore
z_score=abs(zscore(df))
print(df.shape)
df_final=df.loc[(z_score<3).all(axis=1)]
print(df_final.shape)

I am not able to identify that after applying the zscore still i am getting the 0 value in row columns so i am 
working continue on the df dataset.

In [ ]:
sns.pairplot(df)

In [ ]:
df.hist(figsize=(26,26))

In [ ]:
#Now saperating the x and y for prediction the claim amount in the Automobile insurance dataset.


In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(df)
x=pd.DataFrame(x,columns=df.columns)

In [ ]:
x
x.head()

In [ ]:
y=df.iloc[:,-22]
y.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)
y

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
def maxr2_score(regr,x,y):
    max_r_score=0
    for r_state in range(42,100):
        x_train, x_test, y_train, y_test = train_test_split(x, y,random_state = r_state,test_size=0.20)
        regr.fit(x_train,y_train)
        y_pred = regr.predict(x_test)
        r2_scr=r2_score(y_test,y_pred)
        print("r2 score corresponding to ",r_state," is ",r2_scr)
        if r2_scr>max_r_score:
            max_r_score=r2_scr
            final_r_state=r_state
    print("max r2 score corressponding to ",final_r_state," is ",max_r_score)
    return final_r_state

In [ ]:
#Lets use Linear regression and check max r2 score corressponding to different random state
from sklearn.linear_model import LinearRegression
lreg=LinearRegression()
r_state=maxr2_score(lreg,x,y)

In [ ]:
#Lets use grid_search to find optimal value of n_neigbors for KNN model
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
neighbors={"n_neighbors":range(1,30)}
knr=KNeighborsRegressor()
gknr = GridSearchCV(knr,neighbors, cv=5)
gknr.fit(x,y)
gknr.best_params_

In [ ]:
#Lets use KNN regression and check max r2 score corresponding to different random states
knr=KNeighborsRegressor(n_neighbors=11)
r_state=maxr2_score(knr,x,y)

In [ ]:
#Lets check the mean r2 score of both LinearRegression model and knn regression
from sklearn.model_selection import cross_val_score
print("Mean r2 score for Linear Regression :",cross_val_score(lreg,x,y,cv=5,scoring="r2").mean())
print("standard deviation in r2 score for Linear Regression:",cross_val_score(lreg,x,y,cv=5,scoring="r2").std())
print()
print("Mean r2 score for KNN Regression:",cross_val_score(knr,x,y,cv=5,scoring="r2").mean())
print("standard deviation in r2 score for KNN Regression: ",cross_val_score(knr,x,y,cv=5,scoring="r2").std())
#Based on below output Linear Regression is performing well

In [ ]:
#Lets check laso Regression and find best value of alpha
from sklearn.linear_model import Lasso
lsreg=Lasso()
parameters={"alpha":[0.001,0.01,0.1,1]}
clf = GridSearchCV(lsreg, parameters, cv=5)
clf.fit(x,y)
clf.best_params_

In [ ]:
#Lets check max_r2 score whenwe use lasso
lsreg=Lasso(alpha=1)
r_stat=maxr2_score(lsreg,x,y)

In [ ]:
#Lets use cross val score with lasso
print("Mean r2_score for Lasso Regression: ",cross_val_score(lsreg,x,y,cv=5,scoring="r2").mean())
print("standard deviation in r2 score for Lasso Regression: ",cross_val_score(lsreg,x,y,cv=5,scoring="r2").std())
#Based on below output i can say lasso performed just like linear regression

In [ ]:
#Now we will try to use gradient boosting technique
#for getting best set of parameters we will use grid search
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings("ignore")
gbr=GradientBoostingRegressor()
parameters={"learning_rate":[0.001,0.01,0.1,1],"n_estimators":[10,100,500,1000,]}
clr = GridSearchCV(gbr, parameters, cv=5)
clf.fit(x,y)
clf.best_params_

In [ ]:
#Now we will use cross val score to check the mean r2 score and standard deviation
gbr=GradientBoostingRegressor(learning_rate=1,n_estimators=500)
print("Mean r2 score for gradiant boosting Regression: ",cross_val_score(gbr,x,y,cv=5,scoring="r2").mean())
print("standard deviation in r2 score for gradient boosting Regression: ",cross_val_score(gbr,x,y,cv=5,scoring="r2").std())

In [ ]:
#Lets check maximum r2_score corresponding to this
r_state=maxr2_score(gbr,x,y)

In [ ]:
#Till now we have observed that gradiant boosting regression is working well


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y,random_state = 85,test_size=0.20)
gbr=GradientBoostingRegressor()
gbr.fit(x_train,y_train)
y_pred=gbr.predict(x_test)

In [ ]:
#Lets find the rmse and r2_score using sklearn,metrics 
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error 
print("RMSE is: ",np.sqrt(mean_squared_error(y_test,y_pred)))
print("r2_score is: ",r2_score(y_test,y_pred))

In [ ]:
from sklearn.externals import joblib

#save the model as a pickel in a file
joblib.dump(gbr, "Automobile_gbr.pkl")

You will see that in LinearRegression r2 score of 72 is 0.95
In KNeighborsRegression r2 score of 82 is 0.83
In Lasso r2 score of 72 is 0.95
In GradientBoostingRegressor r2 score of 85 is 0.99 
So best among is GradientBoostingRegressor i have save that 

Conclusion : In this dataset firstly i look the data since i have to predict the claim amount in the Automobile insurance dataset so I take it as a Y variable and rest column was taken as a X before spliting the dataset i check the datatype and change the datatype of the columns and see the correlation , null values , description, skewness and pairplot i check the outlier also.Than I split the data into x and y and apply the standard scaler in x and in y the label encoder.

You will see that in LinearRegression r2 score of 72 is 0.95
In KNeighborsRegression r2 score of 82 is 0.83
In Lasso r2 score of 72 is 0.95
In GradientBoostingRegressor r2 score of 85 is 0.99 
So best among is GradientBoostingRegressor i have save that 